### `RandomForest` Modelling on Hyperglycemic data with imbalanced labelling

In [19]:
import numpy as np
import pandas as pd
import warnings

from imblearn.over_sampling import SMOTE

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier


from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectFromModel

from collections import Counter

SEED=90089

## Reading the input data

In [20]:
# Read Data
patient_df = pd.read_csv('../data/hyperglycemic_patients_w_bin_categories.csv', index_col=0)

# print detail summary
print("Dataframe shape:   ", patient_df.shape)
print("Dataframe Features:", patient_df.columns)
patient_df.head()

Dataframe shape:    (13424, 27)
Dataframe Features: Index(['glucose_max', 'anchor_age', 'dod', 'gender', 'dbp_mean', 'sbp_mean',
       'glucose_mean', 'heart_rate_mean', 'spo2_mean', 'resp_rate_mean',
       'temperature_mean', 'apsiii', 'glucose_score', 'los', 'avg_bmi_value',
       'avg_sofa', 'admission_type', 'label', 'AMBULATORY OBSERVATION',
       'DIRECT EMER.', 'DIRECT OBSERVATION', 'ELECTIVE', 'EU OBSERVATION',
       'EW EMER.', 'OBSERVATION ADMIT', 'SURGICAL SAME DAY ADMISSION',
       'URGENT'],
      dtype='object')


,glucose_max,anchor_age,dod,gender,dbp_mean,sbp_mean,glucose_mean,heart_rate_mean,spo2_mean,resp_rate_mean,...,label,AMBULATORY OBSERVATION,DIRECT EMER.,DIRECT OBSERVATION,ELECTIVE,EU OBSERVATION,EW EMER.,OBSERVATION ADMIT,SURGICAL SAME DAY ADMISSION,URGENT
0,277.0,36,0,0,64.320000,112.880000,221.272727,101.800000,92.000000,17.346154,...,0,0,0,0,0,0,0,1,0,0
1,2340.0,35,0,0,77.769231,122.961538,468.592593,105.361111,96.093023,19.813953,...,1,0,0,0,0,0,1,0,0,0
2,259.0,19,1,1,49.300000,74.200000,259.000000,89.750000,70.000000,26.333333,...,0,0,0,0,0,0,1,0,0,0
3,406.0,24,0,0,57.333333,92.848485,292.000000,86.631579,96.162162,15.289474,...,2,0,0,0,0,0,1,0,0,0
4,398.0,31,0,0,78.600000,133.800000,285.000000,100.826087,90.304348,25.173913,...,0,0,0,0,0,0,1,0,0,0


In [6]:
print("Null value in Feature set summary:\n",patient_df.isnull().sum(), "\n","--"*15)
# print("Label Distirbution:\n", patient_df['label'].value_counts())

Null value in Feature set summary:
 glucose_max                    0
anchor_age                     0
dod                            0
gender                         0
dbp_mean                       0
sbp_mean                       0
glucose_mean                   0
heart_rate_mean                0
spo2_mean                      0
resp_rate_mean                 0
temperature_mean               0
apsiii                         0
glucose_score                  0
los                            0
avg_bmi_value                  0
avg_sofa                       0
admission_type                 0
label                          0
AMBULATORY OBSERVATION         0
DIRECT EMER.                   0
DIRECT OBSERVATION             0
ELECTIVE                       0
EU OBSERVATION                 0
EW EMER.                       0
OBSERVATION ADMIT              0
SURGICAL SAME DAY ADMISSION    0
URGENT                         0
dtype: int64 
 ------------------------------


In [9]:
print("Data Cluster Distribution", Counter(patient_df['label'].value_counts()))

Data Cluster Distribution Counter({8365: 1, 3211: 1, 1848: 1})


In [10]:
numerical_features = ['glucose_max', 'anchor_age', 'dbp_mean', 
                    'sbp_mean', 'glucose_mean', 'heart_rate_mean', 'spo2_mean', 
                    'resp_rate_mean', 'temperature_mean', 'apsiii', 'glucose_score', 'avg_bmi_value']

# Standardized variable for consistent measurement across numerical values
scaler = StandardScaler()
X_std = scaler.fit_transform(patient_df[numerical_features])

# Interpolate categorical variables
X_cat = patient_df[['gender', 'AMBULATORY OBSERVATION',
       'DIRECT EMER.', 'DIRECT OBSERVATION', 'ELECTIVE', 'EU OBSERVATION',
       'EW EMER.', 'OBSERVATION ADMIT', 'SURGICAL SAME DAY ADMISSION',
       'URGENT']]


X = np.concatenate([X_std, X_cat.to_numpy()], axis=1)
y = patient_df["label"]
print("X Standardized data Shape: ", X_std.shape)
print("X Categorical data shape:  ", X_cat.shape)
print("X, y shape:                ", X.shape, y.shape)

X Standardized data Shape:  (13424, 12)
X Categorical data shape:   (13424, 10)
X, y shape:                 (13424, 22) (13424,)


In [11]:
# Split data based on training 
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size=0.8, 
                                                    random_state=SEED)
print("Train data shape:        ", X_train.shape, y_train.shape)
print("Test data shape:         ", X_test.shape, y_test.shape)

Train data shape:         (10739, 22) (10739,)
Test data shape:          (2685, 22) (2685,)


In [15]:
# Resample data to tackle class imbalance
oversample = SMOTE()
X_train, y_train = oversample.fit_resample(X_train, y_train)

# summarize the resampled label distribution
print("Train label distribution:", dict(Counter(y_train)))

Train label distribution: {1: 6704, 0: 6704, 2: 6704}


### Hyperparameter Tunning on `RandomForest`

In [70]:
param_grid = {
 'bootstrap'        : [True, False],
 'max_depth'        : [20, 40, 80, 100, None],
 'min_samples_leaf' : [1, 4],
 'max_features'     : ['sqrt', 'log2'],
 'min_samples_split': [2, 5, 10],
 'n_estimators'     : [200, 600, 1000]}
rf_model = RandomForestClassifier(random_state=SEED)

In [71]:
# disable deprecation warnings
warnings.filterwarnings('ignore')

In [72]:
random_grid_rf = RandomizedSearchCV(rf_model, param_distributions=param_grid, 
                                cv=5, n_jobs=8)
random_grid_rf.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=90089),
                   n_jobs=8,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [20, 40, 80, 100, None],
                                        'max_features': ['sqrt', 'log2'],
                                        'min_samples_leaf': [1, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 600, 1000]})

In [73]:
print("----         Results from Grid Search           ---")
print("\n The best estimator across ALL searched params:\n", random_grid_rf.best_estimator_)

## Removed scoring due to combinations yielding NaN results
#  print("\n The best score across ALL searched params:\n", grid_RF.best_score_) 
print("\n The best parameters across ALL searched params:\n", random_grid_rf.best_params_)

----         Results from Grid Search           ---

 The best estimator across ALL searched params:
 RandomForestClassifier(bootstrap=False, max_depth=80, n_estimators=1000,
                       random_state=90089)

 The best parameters across ALL searched params:
 {'n_estimators': 1000, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 80, 'bootstrap': False}


In [16]:
# Instantiate RF classifier
rf_clf = RandomForestClassifier(n_estimators=1000, 
                                min_samples_split=2,
                                min_samples_leaf=1,
                                max_depth=80,
                                max_features='sqrt',
                                bootstrap=False,
                                random_state=SEED,
                                n_jobs=4)

### Evaluation on `RandomForest` with balance data

In [17]:
rf_clf.fit(X_train, y_train)
print("Model Accuracy based on Testing dataset:", rf_clf.score(X_test, y_test))

Model Accuracy based on Testing dataset: 0.6286778398510242


In [18]:
print("RF Feature Importance:", rf_clf.feature_importances_, "\n", "_"*60)

y_pred = rf_clf.predict(X_test)
print(classification_report(y_pred, y_test))

RF Feature Importance: [7.37351237e-02 7.69373785e-02 7.49974415e-02 7.81938076e-02
 7.19113591e-02 7.53682751e-02 7.85158269e-02 7.86100824e-02
 8.23315635e-02 1.58268603e-01 9.88678960e-03 6.18868118e-02
 2.73988629e-02 4.31312247e-06 5.89748179e-03 6.23962884e-05
 2.56425865e-03 8.89908082e-05 1.52203028e-02 1.00708989e-02
 5.15177156e-03 1.28976602e-02] 
 ____________________________________________________________
              precision    recall  f1-score   support

           0       0.84      0.72      0.78      1947
           1       0.22      0.38      0.28       383
           2       0.39      0.40      0.40       355

    accuracy                           0.63      2685
   macro avg       0.49      0.50      0.48      2685
weighted avg       0.69      0.63      0.66      2685



In [56]:
feature_selection = SelectFromModel(rf_clf, prefit=True)
feature_idx = feature_selection.get_support()

#Identifying features after feature selections
feature_df = patient_df[numerical_features+['gender']]
feature_name = feature_df.columns[feature_idx]

print("selected_features:    ", feature_name)

X_new = feature_selection.transform(X)
print("Select Feature.shape: ", X_new.shape)

selected_features:     Index(['glucose_max', 'anchor_age', 'dbp_mean', 'sbp_mean', 'heart_rate_mean',
       'spo2_mean', 'resp_rate_mean', 'temperature_mean', 'apsiii'],
      dtype='object')
Select Feature.shape:  (12872, 9)


In [57]:
# Split data based on training 
X_train, X_test, y_train, y_test = train_test_split(X_new, y, 
                                                    train_size=0.8, 
                                                    random_state=SEED)

# Resample data to tackle class imbalance
oversample = SMOTE()
X_train, y_train = oversample.fit_resample(X_train, y_train)

print("Train data shape:", X_train.shape, y_train.shape)
print("Test data shape:", X_test.shape, y_test.shape)

Train data shape: (20187, 9) (20187,)
Test data shape: (2575, 9) (2575,)


### Fitting `RandomForest` again after feature selections

In [58]:
# Instantiate SVC classifierr
rf_clf = RandomForestClassifier(n_estimators=1400, 
                                min_samples_split=2,
                                min_samples_leaf=1,
                                max_features='log2',
                                max_depth=None,
                                bootstrap=False,
                                random_state=SEED,
                                n_jobs=4)
rf_clf.fit(X_train, y_train)
print("Model Accuracy based on Testing dataset:", rf_clf.score(X_test, y_test))

Model Accuracy based on Testing dataset: 0.610873786407767


In [60]:
y_pred = rf_clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.74      0.77      1760
           1       0.26      0.35      0.30       475
           2       0.35      0.29      0.31       340

    accuracy                           0.61      2575
   macro avg       0.47      0.46      0.46      2575
weighted avg       0.63      0.61      0.62      2575

